# AGLAE data treatment

In [1]:
%pip install ipywidgets

In [2]:
import bae
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import io
import pandas as pd
from pandas.api.typing import DataFrameGroupBy

In [3]:
uploader = widgets.FileUpload()
display(uploader)

FileUpload(value=(), description='Upload')

In [6]:
pd.set_option("display.float_format", "{:.0f}".format)
def pager(df, start, nb):
    return df[start:start+nb]

In [7]:
print(uploader.value)
pige_corrected_compositions_table = pd.read_excel(io.BytesIO(uploader.value[0].content), index_col=[0, 1, 2, 3])
normalized_compositions_table = pige_corrected_compositions_table.apply(bae.normalize, axis=1)

({'name': '240917_compos_normalisees_python.xlsx', 'type': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'size': 1106189, 'content': <memory at 0x000001DBAA719000>, 'last_modified': datetime.datetime(2024, 9, 18, 14, 36, 12, 299000, tzinfo=datetime.timezone.utc)},)


In [10]:
def extract_simplified_name(row):
    name = row[2]
    if "_" in name:
        split = name.rsplit('_', 1)
        return split[0] if "pt" in split[1] or "map" in split[1] else name
    else:
        return name

x_widget = widgets.FloatSlider(min=0.2, max=3.0, step=0.1)
@interact(df=fixed(normalized_compositions_table), tolerance=x_widget, threshold=fixed(10000))
def remove_outliners(df: pd.DataFrame, tolerance, threshold=10000):
    to_delete=[]
    major_group = df[normalized_compositions_table > threshold].groupby(extract_simplified_name, sort=False)
    for name, grp in major_group:
        if grp.count().max() <= 1:
            continue
        median = grp.median()
        d = (grp - median)/median
        for i, row in d.iterrows():
            #print(row)
            for j, elt in row[row>tolerance].items():
                print(i[2], j)
                to_delete.append(i)
                #to_delete.loc[i] = j
    return to_delete

#normalized_compositions_table.loc[to_delete[0]]
#df_group.filter(detect_outliers)
#pager(res,5,10)
#pager(comp_table_without_outliners,90,10)
#for idx in to_delete:
#    print(idx[2])


interactive(children=(FloatSlider(value=0.2, description='tolerance', max=3.0, min=0.2), Output()), _dom_class…

In [16]:
table_without_outliners = normalized_compositions_table.drop(remove_outliners(normalized_compositions_table, tolerance=0.5))


ange_manche_bleufonce_map1 Na2O PIGE
ange_aureole_bleu_moyen_opaque_pt3 PbO
taureau_muffle_vert_translucide_map1 Na2O PIGE
taureau_muffle_jaune_opaque_pt1 Na2O PIGE
taureau_muffle_turquoise_opaque_map1 Na2O PIGE
taureau_cou_point_blanc_opaque_pt1 Sb2O5
taureau_tete_bleu_moyen_opaque_pt2 Sb2O5
taureau_aile_bleu_fonce_translucide_pt2 Fe2O3


In [27]:
final_table_ppm = table_without_outliners.groupby(extract_simplified_name, sort=False).agg(["mean", "std"])
pager(table_without_outliners.groupby(extract_simplified_name, sort=False).agg(["mean", "std"]), 5, 40)

Na2O PIGE         MgO       \
                                                        mean   std  mean  std   
BGIRA 3,  , N° 6 / 6    X: 58168   Y: -22092   ...     48033   NaN 16089  NaN   
ange_manche_bleufonce                                  38898  3557  4036  516   
ange_main_roseopaque                                   15193    11  5148  265   
ange_manche_vert_translucide                           48652  1510 12404  416   
ange_manche_blanc_opaque                               32200  1293 33410 1717   
ange_cape_rouge_fonce_translucide                      42593   578  6320  132   
ange_aureole_blanc_bleute_opaque                       35133  1806  5245  356   
ange_aureole_bleu_moyen_opaque                         45468   336  4355  235   
ange_aile_droite_rouge_clair_opaque                    18003   223  7312  639   
ange_aile_droite_rouge_fonce_opaque                    39771 19764  8211  574   
bandeau_droite_M1_vert_transluide                      52594   NaN 23076  NaN   
bandeau_droite_M1_vert_translucide                     54357  2498 20616  592   
bandeau_droite_M2_bleu_fonce_translucide               38032  2858  4472  346   
bandeau_droite_M2_vert_translucide                     54256   595 20594 1835   
bandeau_droite_M2_blanc_opaque                         39510  1482  9091 8308   
bandeau_droite_M2_rouge_opaque                         19292   584 10656  705   
bandeau_droite_M2_bleu_clair_translucide               40574  1518  4420  650   
bandeau_droite_M2_bleu_turquoise                       46810  1213  4850  476   
bandeau_droite_inscription_emaillee_I5_lettreD         45083 14248  9047  151   
bandeau_droite_inscription_emaillee_I5_lettreE         48988   NaN 10409  NaN   
lion_tete_vert_translucide                             54495   375 11571  609   
lion_oeil_blanc_opaque                                 35902  1976  5073  255   
lion_museau_rouge_fonce_translucide                    39931  1961  6144  507   
lion_nimbe_bleu_fonce_translucide                      40634   940  4278  316   
lion_cou_bleu_turquoise_opaque                         49889  2040  3793  406   
lion_cou_bleu_turquoise_opaque_test                   133144   NaN  6035  NaN   
lion_aile_droite_jaune_opaque                          17465   107  4589  735   
lion_croupe_rouge_clair_opaque                         20449   534 13914  657   
lion_croupe_rouge_clair_opaque_test                    46115   NaN 14630  NaN   
lion_bordure_bleu_moyen                                48154  2212  4043  270   
lion_patte_arriere_gauche_vert_translucide_map_...    138091   NaN 10532  NaN   
lion_patte_arriere_gauche_rouge_translucide_map...    106625   NaN  5361  NaN   
bandeau_droite_M1_vert_translucide_map_test           143731   NaN 19049  NaN   
Brill_A_INA                                           147933   NaN 23039  NaN   
Brill_B_INA                                           152316   NaN  9040  NaN   
Brill_D_INA                                            44106   NaN 34357  NaN   
Brill A,  , N° 1 / 6    X: -30616   Y: 44382   ...    144902   NaN 22611  NaN   
Brill B,  , N° 2 / 6    X: -59056   Y: 42782   ...    161442   NaN  8643  NaN   
Brill C,  , N° 3 / 6    X: -90866   Y: 45592   ...     36029   NaN 20169  NaN   
Brill D                                                46367   NaN 34109  NaN   

                                                   Al2O3        SiO2        \
                                                    mean  std   mean   std   
BGIRA 3,  , N° 6 / 6    X: 58168   Y: -22092   ... 18111  NaN 576904   NaN   
ange_manche_bleufonce                              29020  851 780376  2858   
ange_main_roseopaque                               28306  878 665379  2580   
ange_manche_vert_translucide                       35093  139 714750  1768   
ange_manche_blanc_opaque                           26126 1126 724849  2311   
ange_cape_rouge_fonce_translucide                  32283  678 739572  2711   
ange_aureole_blanc_bleut

In [26]:
major_elements_list = ["Na2O PIGE", "MgO", "Al2O3", "SiO2", "P2O5", "SO3", "Cl", "K2O", "CaO", "TiO2","V2O3", "Cr2O3", "MnO", "Fe2O3", "CoO", "NiO", "CuO", "ZnO", "As2O5", "SrO", "Ag2O", "SnO2", "Sb2O5", "BaO", "Au2O3", "PbO"]

In [29]:
final_table_ppm_major_elements = final_table_ppm[major_elements_list]
pager(final_table_ppm_major_elements,5,40)

Na2O PIGE         MgO       \
                                                        mean   std  mean  std   
BGIRA 3,  , N° 6 / 6    X: 58168   Y: -22092   ...     48033   NaN 16089  NaN   
ange_manche_bleufonce                                  38898  3557  4036  516   
ange_main_roseopaque                                   15193    11  5148  265   
ange_manche_vert_translucide                           48652  1510 12404  416   
ange_manche_blanc_opaque                               32200  1293 33410 1717   
ange_cape_rouge_fonce_translucide                      42593   578  6320  132   
ange_aureole_blanc_bleute_opaque                       35133  1806  5245  356   
ange_aureole_bleu_moyen_opaque                         45468   336  4355  235   
ange_aile_droite_rouge_clair_opaque                    18003   223  7312  639   
ange_aile_droite_rouge_fonce_opaque                    39771 19764  8211  574   
bandeau_droite_M1_vert_transluide                      52594   NaN 23076  NaN   
bandeau_droite_M1_vert_translucide                     54357  2498 20616  592   
bandeau_droite_M2_bleu_fonce_translucide               38032  2858  4472  346   
bandeau_droite_M2_vert_translucide                     54256   595 20594 1835   
bandeau_droite_M2_blanc_opaque                         39510  1482  9091 8308   
bandeau_droite_M2_rouge_opaque                         19292   584 10656  705   
bandeau_droite_M2_bleu_clair_translucide               40574  1518  4420  650   
bandeau_droite_M2_bleu_turquoise                       46810  1213  4850  476   
bandeau_droite_inscription_emaillee_I5_lettreD         45083 14248  9047  151   
bandeau_droite_inscription_emaillee_I5_lettreE         48988   NaN 10409  NaN   
lion_tete_vert_translucide                             54495   375 11571  609   
lion_oeil_blanc_opaque                                 35902  1976  5073  255   
lion_museau_rouge_fonce_translucide                    39931  1961  6144  507   
lion_nimbe_bleu_fonce_translucide                      40634   940  4278  316   
lion_cou_bleu_turquoise_opaque                         49889  2040  3793  406   
lion_cou_bleu_turquoise_opaque_test                   133144   NaN  6035  NaN   
lion_aile_droite_jaune_opaque                          17465   107  4589  735   
lion_croupe_rouge_clair_opaque                         20449   534 13914  657   
lion_croupe_rouge_clair_opaque_test                    46115   NaN 14630  NaN   
lion_bordure_bleu_moyen                                48154  2212  4043  270   
lion_patte_arriere_gauche_vert_translucide_map_...    138091   NaN 10532  NaN   
lion_patte_arriere_gauche_rouge_translucide_map...    106625   NaN  5361  NaN   
bandeau_droite_M1_vert_translucide_map_test           143731   NaN 19049  NaN   
Brill_A_INA                                           147933   NaN 23039  NaN   
Brill_B_INA                                           152316   NaN  9040  NaN   
Brill_D_INA                                            44106   NaN 34357  NaN   
Brill A,  , N° 1 / 6    X: -30616   Y: 44382   ...    144902   NaN 22611  NaN   
Brill B,  , N° 2 / 6    X: -59056   Y: 42782   ...    161442   NaN  8643  NaN   
Brill C,  , N° 3 / 6    X: -90866   Y: 45592   ...     36029   NaN 20169  NaN   
Brill D                                                46367   NaN 34109  NaN   

                                                   Al2O3        SiO2        \
                                                    mean  std   mean   std   
BGIRA 3,  , N° 6 / 6    X: 58168   Y: -22092   ... 18111  NaN 576904   NaN   
ange_manche_bleufonce                              29020  851 780376  2858   
ange_main_roseopaque                               28306  878 665379  2580   
ange_manche_vert_translucide                       35093  139 714750  1768   
ange_manche_blanc_opaque                           26126 1126 724849  2311   
ange_cape_rouge_fonce_translucide                  32283  678 739572  2711   
ange_aureole_blanc_bleut

In [42]:
final_table_wt = (final_table_ppm / 10000).round(3)
pager(final_table_wt,5,40)

Na2O PIGE           MgO  \
                                                        mean    std   mean   
BGIRA 3,  , N° 6 / 6    X: 58168   Y: -22092   ...     4.803    NaN  1.609   
ange_manche_bleufonce                                  3.890  0.356  0.404   
ange_main_roseopaque                                   1.519  0.001  0.515   
ange_manche_vert_translucide                           4.865  0.151  1.240   
ange_manche_blanc_opaque                               3.220  0.129  3.341   
ange_cape_rouge_fonce_translucide                      4.259  0.058  0.632   
ange_aureole_blanc_bleute_opaque                       3.513  0.181  0.525   
ange_aureole_bleu_moyen_opaque                         4.547  0.034  0.436   
ange_aile_droite_rouge_clair_opaque                    1.800  0.022  0.731   
ange_aile_droite_rouge_fonce_opaque                    3.977  1.976  0.821   
bandeau_droite_M1_vert_transluide                      5.259    NaN  2.308   
bandeau_droite_M1_vert_translucide                     5.436  0.250  2.062   
bandeau_droite_M2_bleu_fonce_translucide               3.803  0.286  0.447   
bandeau_droite_M2_vert_translucide                     5.426  0.060  2.059   
bandeau_droite_M2_blanc_opaque                         3.951  0.148  0.909   
bandeau_droite_M2_rouge_opaque                         1.929  0.058  1.066   
bandeau_droite_M2_bleu_clair_translucide               4.057  0.152  0.442   
bandeau_droite_M2_bleu_turquoise                       4.681  0.121  0.485   
bandeau_droite_inscription_emaillee_I5_lettreD         4.508  1.425  0.905   
bandeau_droite_inscription_emaillee_I5_lettreE         4.899    NaN  1.041   
lion_tete_vert_translucide                             5.449  0.037  1.157   
lion_oeil_blanc_opaque                                 3.590  0.198  0.507   
lion_museau_rouge_fonce_translucide                    3.993  0.196  0.614   
lion_nimbe_bleu_fonce_translucide                      4.063  0.094  0.428   
lion_cou_bleu_turquoise_opaque                         4.989  0.204  0.379   
lion_cou_bleu_turquoise_opaque_test                   13.314    NaN  0.603   
lion_aile_droite_jaune_opaque                          1.746  0.011  0.459   
lion_croupe_rouge_clair_opaque                         2.045  0.053  1.391   
lion_croupe_rouge_clair_opaque_test                    4.611    NaN  1.463   
lion_bordure_bleu_moyen                                4.815  0.221  0.404   
lion_patte_arriere_gauche_vert_translucide_map_...    13.809    NaN  1.053   
lion_patte_arriere_gauche_rouge_translucide_map...    10.662    NaN  0.536   
bandeau_droite_M1_vert_translucide_map_test           14.373    NaN  1.905   
Brill_A_INA                                           14.793    NaN  2.304   
Brill_B_INA                                           15.232    NaN  0.904   
Brill_D_INA                                            4.411    NaN  3.436   
Brill A,  , N° 1 / 6    X: -30616   Y: 44382   ...    14.490    NaN  2.261   
Brill B,  , N° 2 / 6    X: -59056   Y: 42782   ...    16.144    NaN  0.864   
Brill C,  , N° 3 / 6    X: -90866   Y: 45592   ...     3.603    NaN  2.017   
Brill D                                                4.637    NaN  3.411   

                                                           Al2O3         \
                                                      std   mean    std   
BGIRA 3,  , N° 6 / 6    X: 58168   Y: -22092   ...    NaN  1.811    NaN   
ange_manche_bleufonce                               0.052  2.902  0.085   
ange_main_roseopaque                                0.027  2.831  0.088   
ange_manche_vert_translucide                        0.042  3.509  0.014   
ange_manche_blanc_opaque                            0.172  2.613  0.113   
ange_cape_rouge_fonce_translucide                   0.013  3.228  0.068   
ange_aureole_blanc_bleute_opaque                    0.036  2.890  0.055   
ange_aureole_bleu_moyen_opaque                      0.023  2.763  0.002   
ange_aile_droite_rouge_c

In [44]:
final_table_wt_major_elts = final_table_wt[major_elements_list]

In [50]:
with pd.ExcelWriter("final.xlsx") as writer:
    final_table_ppm.map(lambda x: f"{x:.0f}").to_excel(writer, sheet_name="ppm all elements")
    final_table_ppm_major_elements.map(lambda x: f"{x:.0f}").to_excel(writer, sheet_name="ppm major elements")
    final_table_wt.to_excel(writer, sheet_name="wt all elements")
    final_table_wt_major_elts.to_excel(writer, sheet_name="wt major elements")